## re-parse the spaadia corpus

remove `pause` and `overlap` XML tags that cause errors in parsing

In [1]:
import codecs
import os
import pandas as pd
import re
import xml.etree.ElementTree as ET

In [2]:
path = 'data/spaadia/'
newpath = 'data/spaadia_edit/'

In [3]:
for filename in os.listdir(path):
    
    with open(path+filename, 'r') as f:
        with open(newpath+filename, 'w') as n:
        
            file = f.readlines()
            for line in file:
#                 if re.findall(r'\<(pause|overlap) [0-9a-z="\s]*\/\>\s', line):
#                     print(line)
                line = re.sub(r'\<(pause|overlap) [0-9a-z="\s]*\/\>\s', '', line)
                n.write(line)

In [4]:
dialogs = []

diagc = 0
for filename in os.listdir(newpath):
    
    tree = ET.parse(newpath+filename)
    dialog = tree.getroot()
    
    filenm = []
    diagno = []
    turnno = []
    uttrno = []
    speakr = []
    usrtxt = []
    punct = []
    snttyp = []
    sp_act = []
    topic = []

    for turn in dialog:

        for ln in turn:
            
            filenm.append(filename)
            turnno.append(turn.attrib['n'])
            uttrno.append(ln.attrib['n'])
            speakr.append(turn.attrib['speaker'])
            snttyp.append(ln.tag)

            if 'sp-act' in ln.attrib:
                sp_act.append(ln.attrib['sp-act'])
            else:
                sp_act.append('none')

            if 'topic' in ln.attrib:
                topic.append(ln.attrib['topic'])
            else:
                topic.append('none')

            p = 'none'
            for w in ln:
                if w.tag == 'punc':
                    p = w.attrib['type']

            punct.append(p)
            usrtxt.append(ln.text.strip())
            
    diagno = [diagc for i in range(len(turnno))]
    
    dialog = pd.DataFrame({
        'filename' : filenm,
        'dialog_num' : diagno,
        'turn_num' : turnno,
        'line_num' : uttrno,
        'speaker' : speakr,
        'message' : usrtxt,
        'punct' : punct,
        'sent_type' : snttyp,
        'speech_act' : sp_act,
        'topic' : topic
    })
    
    dialog = dialog[['filename', 'dialog_num', 'turn_num', 'line_num', 'speaker', 'message',
                    'punct', 'sent_type', 'speech_act', 'topic']]
    
    dialogs.append(dialog)

    diagc += 1

In [5]:
dialogs[0].head()

filename  dialog_num turn_num line_num speaker  \
0  trainline26.xml           0        1        1       A   
1  trainline26.xml           0        1        2       A   
2  trainline26.xml           0        1        3       A   
3  trainline26.xml           0        1        4       A   
4  trainline26.xml           0        2        5       B   

                                             message  punct sent_type  \
0                                     good afternoon   stop      frag   
1                                  Virgin Trainlines  level      frag   
2                                    Sandra speaking   stop      frag   
3  for which journey do you wish to purchase a ti...  query      q-wh   
4                                                 oh   none        dm   

     speech_act    topic  
0         greet     none  
1         refer     none  
2  identifySelf     none  
3     reqDirect  journey  
4       exclaim     none

In [6]:
data = pd.DataFrame([])
for diag in dialogs:
    data = data.append(diag)

In [7]:
data.to_csv('data/spaadia_parsed.csv', index=False)